# Teste Técnico para Estagio - Rafael Geraldini Francisco

#### Primeiro passo foi chamar algumas bibliotecas

In [1]:
import pandas as pd
import numpy as np
from collections import deque

#### Irei definir uma classe Graph para tratar os dados fornecidos, irei assumir que são do tipo "undirect" e de mesmo pesos. Será representado por lista adjacente em um dicionario do python

In [2]:
class Graph:
    def __init__(self, q_vertex):                                    #Inicia a classe
        self.d_q_vertex = q_vertex                                   #Define a variavel interna
        self.d_vertex = range(self.d_q_vertex)                       #Tamanho

        self.d_ladj = {vertex: set() for vertex in self.d_vertex}    #Prepara o set

    def new_edge(self, vertex1, vertex2):                            #Comando para adicionar novo edge
        self.d_ladj[vertex1].add(str(vertex2))                       #Por ser undirect
        self.d_ladj[vertex2].add(str(vertex1))

#    def print_ladj(self):
#        for key in self.d_ladj.keys():
#            print("vertex", key, ": ", self.d_ladj[key])

#### Para calculo da "Closeness Centrality" irei utilizar o algoritmo A*, assim irei definir uma classe com ele onde tudo sera tratado

##### A função heuristica foi considerada igual a 1 para todos os nodes, visto que no calculo manual do caminho para um um grupo de 10 edges encontrou uma distancia media de 2, assim não aparece no calculo

In [3]:
class Astar:
    def __init__(self, ladj):
        self.ladj = ladj

    def l_edges(self, v):
        return self.ladj[v]

    def alg_astar(self, source, target):
        vistos_nexaminados = set([source])      #Visitado mas ainda falta nodes
        vistos_examinados = set([])             #Visitado completamente
        d = {}                                  #Distancia dos nodes
        d[source] = 0                           #Distancia source source
        prox = {}                               #Na proximidade
        prox[source] = source

        while len(vistos_nexaminados) > 0:
            n = None

            for v in vistos_nexaminados:        #Encontra a menor distancia
                if n == None or d[v] < d[n] :
                    n = v;

            if n == target:                     #Se chegou, define o caminho usado
                caminho = []

                while prox[n] != n:
                    caminho.append(n)
                    n = prox[n]

                caminho.append(source)

                caminho.reverse()

                return caminho

            for m in self.l_edges(n):           #Para a proximidade
                if m not in vistos_nexaminados and m not in vistos_examinados:
                    vistos_nexaminados.add(m)
                    prox[m] = n
                    d[m] = d[n] + 1
                else:
                    if d[m] > d[n] + 1:
                        d[m] = d[n] + 1
                        prox[m] = n

                        if m in vistos_examinados:
                            vistos_examinados.remove(m)
                            vistos_nexaminados.add(m)

            
            vistos_nexaminados.remove(n)        #Quando terminou com o vertice
            vistos_examinados.add(n)

        return None

#### Importei o arquivo fornecido como um Dataframe do pandas

In [13]:
df = pd.read_csv('edges', sep=' ',header=None) #Importa o arquivo que deve estar na mesma pasta
edges = len(df)  #Encontra a quantidade de edges
vertices = len(df.groupby(0).agg(list).to_dict().get(1)) #Encontra a quantidade de vertices

#### Inicia o Graph

In [22]:
G = Graph(vertices)

df=np.array(df)   #Transforma o dataframe em um array do numpy
edg= range(edges)
for n in edg:
    G.new_edge(df[n,0],df[n,1])

keys_values = G.d_ladj.items()
Cms = {str(key): value for key, value in keys_values} #Transforma a chave do dicionario em string para uso no A*

#### Inicia o Algoritmo A*

In [43]:
GAstar = Astar(Cms)
temp = 10
tempr = range(temp)
vtc = range(vertices)
CC = []
for x in vtc:
    dist = 0
    for y in vtc:
        dist = dist + len(GAstar.alg_astar(str(x), str(y)))
    CC.append((x,1/dist))
    
    

In [32]:
display(C)

100